# Searching data bases

This notebook shows a few examples of how to search astronomical databases programatically.

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
# import ipywidgets as widgets

In [ ]:
import astropy.units as u
from astropy.coordinates.sky_coordinate import SkyCoord
from astropy.units import Quantity
from astroquery.gaia import Gaia  
from astroquery.simbad import Simbad
from astropy.table import Table

# Simbad

Using Simbad or one of its sub-services such as Vizier is pretty much unavoidable for an astronomer. Astroquery can help you with that (although if you are looking for one object I would go to the web page!):

In [ ]:
c = SkyCoord("05h35m17.3s -05d23m28s", frame='icrs')
r = 2*u.arcmin

result = Simbad.query_region(c, radius=r)
print(result)

# JWST

There are several ways to search for JWST data - you can use the MAST service both online and through astroquery, and search through the ESA JWST archive. Here we search for Trappist 1. 

In [ ]:
from astroquery.esa.jwst import Jwst
Jwst.get_status_messages() # Recommended to do before sending a query

In [ ]:
ra_trappist1 = "23h06m29.368s"
dec_trappist1 = "-05d02m29.04"
coord = SkyCoord(ra=ra_trappist1, dec=dec_trappist1, frame='icrs')

In [ ]:
j = Jwst.cone_search(coordinate=coord, radius=0.5*u.arcmin, async_job=True)

In [ ]:
result = j.get_results()

In [ ]:
len(result['instrument_name'])

### Task: 
If you have time, combine the two approaches to find how many observations have been made of the Red Rectangle planetary nebula with JWST.

In [ ]:
res = Simbad.query_object("Red Rectangle")

In [ ]:
ra_rr = res['RA'][0]
dec_rr = res['DEC'][0]
coord_rr = SkyCoord(ra=ra_rr, dec=dec_rr, unit=(u.hourangle, u.deg), frame='icrs')

In [ ]:
j_rr = Jwst.cone_search(coordinate=coord_rr, radius=0.5*u.arcmin, async_job=True)


In [ ]:
result = j_rr.get_results()
print("Number of observations = {0}".format(len(result)))

# Gaia

## Getting an overview of tables

It is imperative to get to know what tables are available in a database - without that it is impossible to create searches. We would normally read the documentation for the archive, but here is a way to print all tables.

In [ ]:
tables = Gaia.load_tables(only_names=True)
for table in (tables):
    print (table.get_qualified_name())    

what you will see here is that there is a lot of repetition - we see both gaiadr2 and gaiadr3 etc. Clearly this is not all we want. So the best is to go to the [Gaia archive](https://www.cosmos.esa.int/web/gaia-users/archive#) - this link is to the documentation. The full structure is recorded in what is called the data model. 

In the end, however, what you need here is the `gaia_source` table and I will also get this for data release 3 (dr3, currently the latest release). To be concrete, let us search around te centre of the Hyades star cluster. The search will take a bit of time, so we launch it as an asynchronous search and ask for it to be dumped to disk

In [ ]:
search_radius = 6.0/2.0 # In degrees
# The position of the Hyades cluster
res = Simbad.query_object("Hyades")
c = SkyCoord(ra=res['RA'][0], dec=res['DEC'][0], unit=(u.hourangle, u.deg))
ra_pos = c.ra.deg
dec_pos = c.dec.deg

query = "SELECT * FROM gaiadr3.gaia_source WHERE DISTANCE(ra,dec,{0:.5f},{1:.5f}) <{2:.1f}".format(ra_pos, dec_pos, search_radius)
job   = Gaia.launch_job_async(query, dump_to_file=True)
print (job)



In [ ]:
# The tables are stored on disk as VOT tables with peculiar naming. It is a good idea to rename these
# to something a bit more useful - although if you are doing something more complex and important you 
# should store a record of what you did. I have renamed my result to Hyades-Gaia-search.vot.gz  - 
# If you are in Google Colab this is not helpful because the file would be delete each time you restart 
# the Colab but for now it is fine. I also converted it to FITS for speeding up.
#
# If the search above takes too long (quite possible!), then uncomment the lines below and download this file.
#
# !wget --quiet -O Hyades-Gaia-search.fits https://www.dropbox.com/scl/fi/90kevuc9oa8q9kbny7t9q/Hyades-Gaia-search.fits?rlkey=d65ewv7gxw1vni6scu75r3nx2&dl=0
# t = Table().read('Hyades-Gaia-search.fits')

The Hyades are at at a distance of $\sim 47$pc, select sources close to this distance and create an approximate HR diagram for these stars.